In [1]:
## Loading standard modules and libraries 

import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt

import keras 
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
from skimage.transform import resize

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data

def check_dicom(filename): 
    
    print('Loading file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)   
    
    if (ds.BodyPartExamined !='CHEST') | (ds.Modality !='DX') | (ds.PatientPosition not in ['PA', 'AP']):
        print('The image is not valid because the image position, the image type or the body part is not as per standards')
        return
    else:
        print('ID:', ds.PatientID, 
              'Age:', ds.PatientAge, 
              'Modality:', ds.Modality,
              'Postion: ', ds.PatientPosition, 
              'Body Part: ', ds.BodyPartExamined, 
              'Study Desc: ', ds.StudyDescription)
    
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input

def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    
    img = resize(img, (224,224))   
    img = img / 255.0  
    grey_img = (img - img_mean) / img_std 
    
    proc_img = np.zeros((224,224,3))
    proc_img[:, :, 0] = grey_img
    proc_img[:, :, 1] = grey_img
    proc_img[:, :, 2] = grey_img
    
    proc_img = np.resize(proc_img, img_size)
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 

def load_model(model_path, weight_path):
    # todo
    
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model

def predict_image(model, img, thresh): 
    # todo    
    
    result = model.predict(img)  
    print('Predicted value:', result)
    
    predict=result[0]
    prediction = "Negative"
    if(predict > thresh):
        prediction = "Positive"
    
    return prediction 

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model2.json" #path to saved model
weight_path = "xray_class_my_model2.best.hdf5" #path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0.49262813   # mean image value from Build and train model line 22
img_std = 0.24496286 # loads the std dev from Build and train model line 22

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.62786263 #threshold value for New Model2 from Build and train model line 66 at 80% Precision 


# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print('Model Classification:', pred , 'for Pneumonia' )
    print('--------------------------------------------------------------------------------------------------------')

Loading file test1.dcm ...
ID: 2 Age: 81 Modality: DX Postion:  PA Body Part:  CHEST Study Desc:  No Finding
Predicted value: [[0.4775539]]
Model Classification: Negative for Pneumonia
--------------------------------------------------------------------------------------------------------
Loading file test2.dcm ...
ID: 1 Age: 58 Modality: DX Postion:  AP Body Part:  CHEST Study Desc:  Cardiomegaly
Predicted value: [[0.47687072]]
Model Classification: Negative for Pneumonia
--------------------------------------------------------------------------------------------------------
Loading file test3.dcm ...
ID: 61 Age: 77 Modality: DX Postion:  AP Body Part:  CHEST Study Desc:  Effusion
Predicted value: [[0.47764364]]
Model Classification: Negative for Pneumonia
--------------------------------------------------------------------------------------------------------
Loading file test4.dcm ...
The image is not valid because the image position, the image type or the body part is not as per sta

Threshold of 0.62786263 is considered at 80% Precision